<a href="https://colab.research.google.com/github/Leriloe/Dataton/blob/main/ntbk_manipulacion_Datos_Dataton_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Técnicas de Manipulación y Transformación de Datos para Análisis


** Herramientas de Analisis de Presupuestario y Transparencia Financiera impartida por: 

Gabriel Purón Cid - Profesor de la División de Administración Pública del Centro de Investigación y Docencia Económicas (CIDE)

### [LINK](https://www.youtube.com/watch?v=ziV_SXy1H0A)

## Obtencion de Datos por medio de API

Comenzamos importando nuestras librerías con las que trabajaremos este laboratorio:

In [1]:
# Librería para pedir datos a la API de Datos
import requests

# Librería para manipulación de arreglos 2D 
import pandas as pd

En la plataforma [Plataforma de Datos Abiertos del Gobierno de la Ciudad de Mexico](https://datos.cdmx.gob.mx/) tenemos dos manera de trabajar con nuestras bases de datos. 

*   Descargando nuestro archivo .xlsx

*   Accediendo a la API (Application Programming Interface)

** ¿Cómo analizar el presupuesto de la CDMX? - Carlos Aviud Vázquez Vidal - Investigador Jr. en el CIEP

[LINK](https://www.youtube.com/watch?v=bXb7nivkTNE)

** Pensar el APIS: Transformar tareas a programas - Andrés Barrios Arenas - Director del Centro de Estudios Aplicados 

[LINK](https://www.youtube.com/watch?v=4GEzGatJPro)




In [2]:
# URL de acceso a la API DATOS CDMX

api_url = 'https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=5c6fb123-5a2f-4ac3-b033-48e12f59973c&limit=15000'

In [3]:
# Hacemos una requisición de los datos a estudiar (conexión a API)

response = requests.get(api_url)

# Revisamos el código de estado de nuestra API, buscámos un 200

print(response.status_code)

data_response = response.json()

# Normalizamos ('aplanamos') nuesto .json y convertimos a un pd.DataFrame
data_1 = pd.json_normalize(data_response['result']['records'])

200


In [4]:
data = data_1.copy()

## Exploración de datos

Nos daremos a la tarea de reconocer la estrucutra y características de los datos que conforman nuestra base a estudiar

In [5]:
# Obtenenemos las dimensiones de la información obtenida 
# (filas, columnas)

data.shape

(15000, 64)

In [6]:
# Imprimimos las primeras 2 filas de nuestra infomación para familiarizarnos
# .tail() = últimas filas

data.head(2)

,_id,clave_presupuestaria,ciclo,periodo,gobierno_general,desc_gobierno_general,sector,desc_sector,subsector,desc_subsector,unidad_responsable,desc_unidad_responsable,finalidad,desc_finalidad,funcion,desc_funcion,subfuncion,desc_subfuncion,area_funcional,desc_area_funcional,modalidad,desc_modalidad,programa_presupuestario,desc_programa_presupuestario,eje,desc_eje,objetivo_desarrollo_sostenible,desc_objetivo_desarrollo_sostenible,tipo_recurso,desc_tipo_recurso,fuente_financiamiento,desc_fuente_financiamiento,fuente_generica,desc_fuente_generica,fuente_especifica,desc_fuente_especifica,ciclo_original,origen_recurso,desc_origen_recurso,fondo,desc_fondo,gasto_programable,desc_gasto_programable,tipo_gasto,desc_tipo_gasto,capitulo,desc_capitulo,concepto,desc_concepto,partida_generica,desc_partida_generica,partida_especifica,desc_partida_especifica,consolidado,desc_consolidado,detalle_consolidado,desc_detalle_consolidado,proyecto_inv,desc_proy_inv,tipo_proyecto,desc_tipo_proyecto,monto_aprobado,monto_modificado,monto_ejercido
0,1,802CD0718520115O28027111200,2018,Aprobado,1,Poder Ejecutivo,2,Gobierno,CD,Alcaldías,02CD07,Alcaldía Gustavo A. Madero,1,Gobierno,18,Otros Servicios Generales,185,Otros,185201,apoyo administrativo,,,,,5.0,"Efectividad, Rendición de Cuentas y Combate a ...",,,1.0,Recursos de libre disposición y financiamientos,5.0,Recursos Federales,15O,Participaciones a Entidades Federativas y Muni...,15O2,Fondo General de Participaciones,2018.0,0.0,Identifica al presupuesto aprobado a la ur med...,15O280,No Etiquetado Recursos Federales-Participacion...,1,Programable,1,Gasto corriente,2000,Materiales y suministros,2700,"Vestuario, blancos, prendas de proteccion y ar...",2710,Vestuario y uniformes,2711,Vestuario y uniformes,2,Si,0,Gasto normal,,,,,5000000.0,,
1,2,802CD0718520115O28031711100,2018,Aprobado,1,Poder Ejecutivo,2,Gobierno,CD,Alcaldías,02CD07,Alcaldía Gustavo A. Madero,1,Gobierno,18,Otros Servicios Generales,185,Otros,185201,apoyo administrativo,,,,,5.0,"Efectividad, Rendición de Cuentas y Combate a ...",,,1.0,Recursos de libre disposición y financiamientos,5.0,Recursos Federales,15O,Participaciones a Entidades Federativas y Muni...,15O2,Fondo General de Participaciones,2018.0,0.0,Identifica al presupuesto aprobado a la ur med...,15O280,No Etiquetado Recursos Federales-Participacion...,1,Programable,1,Gasto corriente,3000,Servicios generales,3100,Servicios basicos,3170,"Servicios de acceso de Internet, redes y proce...",3171,"Servicios de acceso de Internet, redes y proce...",1,No,0,Gasto normal,,,,,3125000.0,,


In [7]:
# Desplegamos información de nuestros datos
# Usamos memory_usage='deep' para conocer la cantidad memoria en uso


data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 64 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   _id                                  15000 non-null  int64  
 1   clave_presupuestaria                 15000 non-null  object 
 2   ciclo                                15000 non-null  int64  
 3   periodo                              15000 non-null  object 
 4   gobierno_general                     15000 non-null  int64  
 5   desc_gobierno_general                15000 non-null  object 
 6   sector                               15000 non-null  int64  
 7   desc_sector                          15000 non-null  object 
 8   subsector                            15000 non-null  object 
 9   desc_subsector                       15000 non-null  object 
 10  unidad_responsable                   15000 non-null  object 
 11  desc_unidad_responsable     

In [9]:
# Revisamos columnas con datos nulos: que sume cuántas filas son de datos nulos (dándole valor a cero). 

nan_cols = data.isna().sum()
nan_cols[nan_cols>0]
# dict(data.isnull().sum())

eje                        10
tipo_recurso             2295
fuente_financiamiento    2295
ciclo_original           2295
origen_recurso           2295
monto_aprobado           2295
dtype: int64

## Limpieza de Datos 

*   Identificación de atributos de interés
*   Reemplazo de valores 
*   Cambio de tipo de dato

In [10]:
# Explorando datos únicos por atributo

data.desc_tipo_proyecto.unique()

array(['', 'Obras', 'Adquisiciones', 'Integrales'], dtype=object)

### Tratando Valores Nulos

Encontramos valores no declarados y los tratamos para poder reconocerlos en nuestro análisis

In [11]:
# Sustituyendo valores NaN por 0 y cambiando el tipo de dato

data.ciclo_original = data.ciclo_original.fillna(value = 0)
data.ciclo_original = data.ciclo_original.astype('int')

In [12]:
# Hacemos un conteo de valores para revisar nuestros cambios

data.ciclo_original.value_counts()

2018    12504
0        2295
2017      184
2016       14
2015        3
Name: ciclo_original, dtype: int64

In [13]:
# Para este ejercicio tomamos atributos categóricos para entender la información que conforma nuestra Base de Datos
# .copy() - para no alterar nuestra base de datos original (indices y datos)
data_desc = data[['_id',\
                  'ciclo',\
                  'periodo',\
                  'desc_gobierno_general',\
                  'desc_sector',\
                  'desc_subsector',\
                  'desc_unidad_responsable',\
                  'desc_finalidad',\
                  'desc_funcion',\
                  'desc_area_funcional',\
                  'desc_eje',\
                  'desc_objetivo_desarrollo_sostenible',\
                  'desc_tipo_recurso',\
                  'desc_fuente_financiamiento',\
                  'desc_fuente_generica',\
                  'desc_fuente_especifica',\
                  'ciclo_original',\
                  'desc_capitulo',\
                  'desc_concepto',\
                  'desc_partida_generica',\
                  'desc_partida_especifica',\
                  'desc_consolidado',\
                  'desc_detalle_consolidado',\
                  'desc_proy_inv',\
                  'desc_tipo_proyecto',\
                  'monto_aprobado',\
                  'monto_ejercido']].copy()

In [14]:
# Al analizar nuestra base de datos localizamos un valor declarado en diversos atributos como "" (valor vacio), mismo que se sustituye por "No Declarado **"

data_desc = data_desc.replace("","No Declarado **")

In [15]:
# El atributo desc_monto_ejercido cuenta con este tipo de valores asi que para ejecutar agregaciones matemáticas debemos sustituir esos valores por "0"

data_desc.monto_ejercido = data_desc.monto_ejercido.replace("No Declarado **", "0")

Dada la naturaleza y fuente de nuestros datos debemos asegurarnos que nuestro tipo de dato nos permite realizar agregaciones matematicas (float, int, date) o procesos de catergorizacion (objects, strings) 

In [18]:
# Convertimos nuestro atributo desc_monto_ejercido a número con decimales (float)

data_desc.monto_ejercido = data_desc.monto_ejercido.astype('float')


In [17]:
# Comparamos las dimensiones de nuetros datos originales y los que estudiaremos en este caso

print(data.shape, data_desc.shape)

(15000, 64) (15000, 27)


# Manipulación de Bases de Datos

*   Reordenamiento de Datos
*   Filtrado Condicional
*   Categorización



In [19]:
# Renombrando columnas

# Opción 1

# Modificamos el nombre de nuestros atributos dado a que estamos ahora tratando nuestra base data_desc 

data_desc.columns = [item[5:] if item.startswith('desc_') else item for item in data_desc.columns]

In [20]:
# Opción 2
# Declaración manual (comentario;)
'''
data_rename = data.rename(columns={'_id' : 'ID',\
                            'desc_gobierno_general' : 'Gobierno_Gral',\
                            'desc_sector' : 'Sector'})

''';

In [21]:
data_desc.columns 

Index(['_id', 'ciclo', 'periodo', 'gobierno_general', 'sector', 'subsector',
       'unidad_responsable', 'finalidad', 'funcion', 'area_funcional', 'eje',
       'objetivo_desarrollo_sostenible', 'tipo_recurso',
       'fuente_financiamiento', 'fuente_generica', 'fuente_especifica',
       'ciclo_original', 'capitulo', 'concepto', 'partida_generica',
       'partida_especifica', 'consolidado', 'detalle_consolidado', 'proy_inv',
       'tipo_proyecto', 'monto_aprobado', 'monto_ejercido'],
      dtype='object')

In [22]:
# Reordenamiento de Columnas

orden_actualizado = ['_id', 'ciclo', 'ciclo_original', 'finalidad', 'fuente_especifica', 'periodo', 'unidad_responsable', 'gobierno_general', 'subsector',\
                     'monto_aprobado', 'monto_ejercido', 'sector', 'funcion', 'area_funcional', 'eje','objetivo_desarrollo_sostenible',\
                     'tipo_recurso', 'fuente_financiamiento', 'fuente_generica', 'capitulo', 'concepto', 'partida_generica',\
                     'partida_especifica', 'consolidado', 'detalle_consolidado', 'proy_inv', 'tipo_proyecto']

data_desc = data_desc[orden_actualizado]

data_desc.head(2)

,_id,ciclo,ciclo_original,finalidad,fuente_especifica,periodo,unidad_responsable,gobierno_general,subsector,monto_aprobado,monto_ejercido,sector,funcion,area_funcional,eje,objetivo_desarrollo_sostenible,tipo_recurso,fuente_financiamiento,fuente_generica,capitulo,concepto,partida_generica,partida_especifica,consolidado,detalle_consolidado,proy_inv,tipo_proyecto
0,1,2018,2018,Gobierno,Fondo General de Participaciones,Aprobado,Alcaldía Gustavo A. Madero,Poder Ejecutivo,Alcaldías,5000000.0,0.0,Gobierno,Otros Servicios Generales,apoyo administrativo,"Efectividad, Rendición de Cuentas y Combate a ...",No Declarado **,Recursos de libre disposición y financiamientos,Recursos Federales,Participaciones a Entidades Federativas y Muni...,Materiales y suministros,"Vestuario, blancos, prendas de proteccion y ar...",Vestuario y uniformes,Vestuario y uniformes,Si,Gasto normal,No Declarado **,No Declarado **
1,2,2018,2018,Gobierno,Fondo General de Participaciones,Aprobado,Alcaldía Gustavo A. Madero,Poder Ejecutivo,Alcaldías,3125000.0,0.0,Gobierno,Otros Servicios Generales,apoyo administrativo,"Efectividad, Rendición de Cuentas y Combate a ...",No Declarado **,Recursos de libre disposición y financiamientos,Recursos Federales,Participaciones a Entidades Federativas y Muni...,Servicios generales,Servicios basicos,"Servicios de acceso de Internet, redes y proce...","Servicios de acceso de Internet, redes y proce...",No,Gasto normal,No Declarado **,No Declarado **


In [23]:
data_desc.ciclo.unique()

array([2018, 2019])

In [24]:
# Filtrando datos por columna

data_gob_2018 = data_desc[(data_desc['ciclo_original'] == 2018 & 2019) & (data_desc['periodo'] == 'Aprobado') & (data_desc['monto_aprobado'] != 0)]

In [25]:
data_gob_2018.shape

(10260, 27)

In [26]:
data_gob_2018.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10260 entries, 0 to 12706
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   _id                             10260 non-null  int64  
 1   ciclo                           10260 non-null  int64  
 2   ciclo_original                  10260 non-null  int64  
 3   finalidad                       10260 non-null  object 
 4   fuente_especifica               10260 non-null  object 
 5   periodo                         10260 non-null  object 
 6   unidad_responsable              10260 non-null  object 
 7   gobierno_general                10260 non-null  object 
 8   subsector                       10260 non-null  object 
 9   monto_aprobado                  10260 non-null  float64
 10  monto_ejercido                  10260 non-null  float64
 11  sector                          10260 non-null  object 
 12  funcion                         

In [27]:
# Modificando el tipo de Dato a tipo fecha 

data_gob_2018.ciclo_original = pd.to_datetime(data_gob_2018.ciclo_original, format = '%Y', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [28]:
data_gob_2018.ciclo = pd.to_datetime(data_gob_2018.ciclo, format = '%Y', errors='ignore')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [29]:
# Categorización 

# Declaramos nuestras etiquetas

montos_aprobados = ['Bajo', 'Moderado', 'Medio', 'Medio Alto', 'Alto']

# Utilizamos la funcion pd.cut() para convertir nuestros valores a intervalos discretos. Considerando valores min y max

grupos = pd.cut(data_gob_2018.monto_aprobado, 5, labels= montos_aprobados)

In [30]:
grupos.value_counts()

Bajo          10252
Moderado          6
Alto              1
Medio             1
Medio Alto        0
Name: monto_aprobado, dtype: int64

In [31]:
grupos.head()

0    Bajo
1    Bajo
2    Bajo
4    Bajo
5    Bajo
Name: monto_aprobado, dtype: category
Categories (5, object): ['Bajo' < 'Moderado' < 'Medio' < 'Medio Alto' < 'Alto']

In [32]:
# La funcion pd.qcut() (Quantile-Based Discretization) 
# categoriza los valores intentando colocar la misma cantidad de datos en cada categoria

grupos_qcut = pd.qcut(data_gob_2018.monto_aprobado, 5, labels= montos_aprobados)

In [33]:
grupos_qcut.value_counts()

Moderado      2100
Bajo          2071
Alto          2052
Medio Alto    2052
Medio         1985
Name: monto_aprobado, dtype: int64

In [34]:
# Obteniendo cuantiles

data_gob_2018.monto_aprobado.describe(percentiles=[x/100 for x in range(0,100,25)])

count    1.026000e+04
mean     6.322303e+06
std      6.625709e+07
min      1.000000e+00
0%       1.000000e+00
25%      3.170600e+04
50%      1.867360e+05
75%      1.125840e+06
max      4.579769e+09
Name: monto_aprobado, dtype: float64

In [35]:
cuts = [1, 31706, 186736, 1125840, 500000000 ,4579769000]
bins = pd.cut(data_gob_2018.monto_aprobado, cuts, labels= montos_aprobados)
bins.value_counts()

Medio         2565
Moderado      2565
Bajo          2564
Medio Alto    2544
Alto            21
Name: monto_aprobado, dtype: int64

In [36]:
# One Hot Encoding - pd.get_dummies
# Convertimos variables categoricas a valiables indicativas (es o no es)

dummies_sector = pd.get_dummies(data_gob_2018['sector'])
dummies_sector.head()

,Ciencia y Tecnología,Consejería Jurídica,Contraloría General,Cultura,Desarrollo Económico,Desarrollo Rural y equidad para las comunidades,Desarrollo Social,Desarrollo Urbano y Vivienda,Educación,Finanzas,Gobierno,Medio Ambiente,Obras y Servicios,Oficialía Mayor,Poder Judicial,Poder Legislativo,Procuraduría Ambiental y del Ordenamiento Territorial,Procuraduría General de Justicia,Protección Civil,Salud,Seguridad Ciudadana,Subsidios,Transportes y Vialidad,Turismo,Órganos Autónomos
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
sector_fuente_esp = pd.concat([data_gob_2018['fuente_especifica'], dummies_sector], axis=1)

In [38]:
sector_fuente_esp.head()

,fuente_especifica,Ciencia y Tecnología,Consejería Jurídica,Contraloría General,Cultura,Desarrollo Económico,Desarrollo Rural y equidad para las comunidades,Desarrollo Social,Desarrollo Urbano y Vivienda,Educación,Finanzas,Gobierno,Medio Ambiente,Obras y Servicios,Oficialía Mayor,Poder Judicial,Poder Legislativo,Procuraduría Ambiental y del Ordenamiento Territorial,Procuraduría General de Justicia,Protección Civil,Salud,Seguridad Ciudadana,Subsidios,Transportes y Vialidad,Turismo,Órganos Autónomos
0,Fondo General de Participaciones,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fondo General de Participaciones,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fondo General de Participaciones,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fondo General de Participaciones,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Fondo General de Participaciones,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Exploración Visual de Datos 04/10/2121

[LINK](https://www.youtube.com/watch?v=4vcLbPIEJng)

*   Gráficos de Caja
*   Gráficos de Violin
*   Histogramas
*   Gráficos de Barra

